**Generating Elasticsearch Top-k Results for Re-ranking**

In [1]:
!pip3 install elasticsearch

In [2]:
!pip3 install elasticsearch-dsl

In [3]:
from elasticsearch import Elasticsearch

import sys
sys.path.insert(0, '../../../BERT-FAQ/')

# import required classes and functions
from reranker import ReRanker
from evaluation import get_relevance_label_df

# import utility functions
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs

In [4]:
# Generate list of test queries, relevance labels for ReRanker class
query_answer_pair_filepath = '../../../BERT-FAQ/data/StackFAQ/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pair_filepath)
test_queries = relevance_label_df[relevance_label_df['query_type'] == 'user_query'].question.unique()

In [5]:
# check first 5 occurences for test_queries
test_queries[:5]

array(['How to make font strikethrough on github.',
       'Is it possible to get  strikethrough letter formatting on github markdown.',
       'Making the text on github crossed out.',
       'Introducing stikethrough formatting on markdown for github.',
       'The <s> tag for font on github markdown doesnt work, is there an alternative?'],
      dtype=object)

In [6]:
# check relevance_label_df
relevance_label_df

,id,question,answer,query_type,label
0,1,How to make font strikethrough on github.,Use ~~tildes around the words~~.,user_query,1
1,2,Is it possible to get strikethrough letter fo...,Use ~~tildes around the words~~.,user_query,1
2,3,Making the text on github crossed out.,Use ~~tildes around the words~~.,user_query,1
3,4,Introducing stikethrough formatting on markdow...,Use ~~tildes around the words~~.,user_query,1
4,5,The <s> tag for font on github markdown doesnt...,Use ~~tildes around the words~~.,user_query,1
...,...,...,...,...,...
7898,7899,How do I get an m dash in google docs?,"On Mac, just hold fn + option + -",user_query,1
7899,7900,Do google docs support the longer style dash (...,"On Mac, just hold fn + option + -",user_query,1
7900,7901,I desperately need the em dash in google docs ...,"On Mac, just hold fn + option + -",user_query,1
7901,7902,"I require the longer (em) dash in google docs,...","On Mac, just hold fn + option + -",user_query,1


In [7]:
# Define instance of ReRanker class
r = ReRanker(
    bert_model_path='', 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [8]:
# create output path to save Elasticsearch top-k results
output_path = "../../../BERT-FAQ/data/StackFAQ/rank_results/unsupervised"
make_dirs(output_path)

In [9]:
# Select top-k value
top_k = 100

In [10]:
# Get top-k Elasticsearch results 

es = Elasticsearch([{'host':'localhost','port':9200}], http_auth=('username', 'password')) 

es_query_by_question = r.get_es_topk_results(es=es, index='stackfaq', query_by=['question'], top_k=top_k)
es_query_by_answer = r.get_es_topk_results(es=es, index='stackfaq', query_by=['answer'], top_k=top_k)
es_query_by_question_answer = r.get_es_topk_results(es=es, index='stackfaq', query_by=['question', 'answer'], top_k=top_k)
es_query_by_question_answer_concat = r.get_es_topk_results(es=es, index='stackfaq', query_by=['question_answer'], top_k=top_k)

2021-03-02 12:53:05 - Generating ES top-k results ...
  0%|          | 0/1249 [00:00<?, ?it/s]2021-03-02 12:53:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.016s]
2021-03-02 12:53:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:53:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.008s]
2021-03-02 12:53:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.011s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.011s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.005s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.009s]
  1%|          | 9/1249 [00:00<00:14, 84.68it/s]2021-03-02 1

2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.008s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.006s]
  7%|▋         | 86/1249 [00:00<00:13, 88.36it/s]2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.009s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.009s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.008s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 

2021-03-02 12:53:07 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:53:07 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:53:07 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:53:07 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:53:07 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.009s]
2021-03-02 12:53:07 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:07 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
 14%|█▍        | 172/1249 [00:01<00:13, 81.34it/s]2021-03-02 12:53:07 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.011s]
2021-03-02 12:53:07 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:07 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02

 20%|█▉        | 248/1249 [00:02<00:11, 88.70it/s]2021-03-02 12:53:08 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.008s]
2021-03-02 12:53:08 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:08 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.009s]
2021-03-02 12:53:08 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.008s]
2021-03-02 12:53:08 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:08 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.009s]
2021-03-02 12:53:08 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.011s]
2021-03-02 12:53:08 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.011s]
2021-03-02 12:53:08 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
 21%|██        | 257/1249 [00:02<00:11, 88.74it/s]2021-03-02 12:53:08 - POST http://localhost:9200/stack

 33%|███▎      | 412/1249 [00:04<00:09, 86.53it/s]2021-03-02 12:53:10 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.007s]
2021-03-02 12:53:10 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.009s]
2021-03-02 12:53:10 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.009s]
2021-03-02 12:53:10 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.007s]
2021-03-02 12:53:10 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.009s]
2021-03-02 12:53:10 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.011s]
2021-03-02 12:53:10 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:10 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:10 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.010s]
2021-03-02 12:53:10 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.007s]
 34%|███▍ 

...

2021-03-02 12:54:01 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.015s]
2021-03-02 12:54:01 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:01 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
 71%|███████   | 881/1249 [00:12<00:06, 60.28it/s]2021-03-02 12:54:01 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:54:01 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.015s]
2021-03-02 12:54:01 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:54:01 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:54:01 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:54:01 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:54:01 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
 71%|█████

2021-03-02 12:54:02 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:54:02 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:54:02 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:54:02 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:02 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:02 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
 77%|███████▋  | 965/1249 [00:13<00:04, 64.92it/s]2021-03-02 12:54:02 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.015s]
2021-03-02 12:54:02 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:54:02 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:54:02 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02

2021-03-02 12:54:03 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:54:03 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.015s]
 84%|████████▎ | 1043/1249 [00:15<00:03, 67.05it/s]2021-03-02 12:54:03 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:03 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:03 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.015s]
2021-03-02 12:54:03 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.015s]
2021-03-02 12:54:03 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:54:04 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:04 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
 84%|████████▍ | 1050/1249 [00:15<00:02, 66.34it/s]2021-03-02 12:54:04 - POST http://localhost:9200/sta

 90%|████████▉ | 1123/1249 [00:16<00:01, 70.33it/s]2021-03-02 12:54:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:54:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.011s]
2021-03-02 12:54:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.011s]
2021-03-02 12:54:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:54:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:54:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.014s]
2021-03-02 12:54:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
 91%|█████████ | 1131/1249 [00:16<00:01, 70.52it/s]2021-03-02 12:54:05 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:05 - POST http://localhost:9200/sta

 96%|█████████▋| 1205/1249 [00:17<00:00, 70.56it/s]2021-03-02 12:54:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-03-02 12:54:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.011s]
2021-03-02 12:54:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.015s]
2021-03-02 12:54:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.016s]
 97%|█████████▋| 1213/1249 [00:17<00:00, 68.87it/s]2021-03-02 12:54:06 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.013s]
2021-03-02 12:54:06 - POST http://localhost:9200/sta

In [11]:
# Save Elasticsearch results to json files
dump_to_json(es_query_by_question, output_path + '/es_query_by_question.json')
dump_to_json(es_query_by_answer, output_path + '/es_query_by_answer.json')
dump_to_json(es_query_by_question_answer, output_path + '/es_query_by_question_answer.json')
dump_to_json(es_query_by_question_answer_concat, output_path + '/es_query_by_question_answer_concat.json')